In [8]:
import BubbleCount.image_preprocess as image_preprocess
import BubbleCount.csv_helpers as csv_helpers
from BubbleCount.post_processing import get_normalized_mse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw
import csv
import psutil
import os
%matplotlib inline

In [9]:
from BubbleCount.counting_model import CountingPipe
model = CountingPipe()  # Only really need the count_hybrid() function in this notebook

===> Using CPU mode.



In [10]:
args = {
    "sample_path": "./Exemplars/",
    "target_path": "./2025/Targets",
    "result_path": "./2025/Outputs/out.csv",
    "output_dir": "./2025/Outputs/",
    "model_path": "./data/pretrainedModels/FamNet_Save1.pth",
    "raw_img_dir": "./2025/Images/SEN10_1.8_6_300-400"
}

In [11]:
# Crop images to AOI
# image_preprocess.crop_to_interest(image_dir=args["raw_img_dir"],region=[65, 770, 1090, 970],output_dir=args["target_path"])

In [12]:
def simplify_target_name(target):
    temp = target.split('_')[1:-1]
    temp = '_'.join(temp)
    serial_number = temp.split('.')[-1]
    batch_name = '.'.join(temp.split('.')[0:-2])

    return f"{batch_name}_{serial_number}"

In [ ]:
result_to_csv = []

# Memory leak tracker

# Load exemplars and targets
Exemplars = image_preprocess.load_exemplars_from_directory(args["sample_path"], reverse_bbox=False)
Targets = image_preprocess.load_target_images_from_directory(args["target_path"])

sample_image = Exemplars[1]

csv_helpers.backup_and_clear_csv(args["result_path"])

process = psutil.Process(os.getpid())


with open(args["result_path"], 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Exemplar", "num_exemps", "rev_bbox", "Target", "Count"])

    base_memory_usage = process.memory_info().rss
    last_mem = base_memory_usage
    
    for target_image in Targets:
        num_exemps = 4
        # Creating the hybrid
        hybrid, hybrid_boxes = image_preprocess.insert_cropped(sample_image['image'], target_image['image'], sample_image['box'], num_exemps)
        target_name = f"{simplify_target_name(target_image['file_name'])}_{num_exemps}exemps"

        # counting
        hybrid_count = model.count_hybrid_and_visualize(hybrid, hybrid_boxes, sample_image["file_name"], target_name,output_directory=args["output_dir"])

        result_to_csv.append([sample_image['file_name'], num_exemps, False, target_name, hybrid_count])

        # append to csv
        writer.writerows(result_to_csv)

        # track memory leaks
        memory_usage = process.memory_info().rss
        loop_memory_usage = memory_usage - base_memory_usage

        mem_diff = loop_memory_usage - last_mem

        last_mem = loop_memory_usage

        print(f"Memory Gain Since Starting:{loop_memory_usage} (Difference: {mem_diff})")

print(f"The counts are saved to {args['result_path']}.")

===> The predicted count for SEN10_1.8_6_300-400_Test_000000_4exemps is:  76.23
===> Visualized output of batchSEN10_1.8_6_300-400_Test_000000_4exemps is saved to ./2025/Outputs/SEN10_1.8_6_300-400_Test_000000_4exemps_out.png
Memory Gain Since Starting:42463232 (Difference: -994234368)
===> The predicted count for SEN10_1.8_6_300-400_Test_000001_4exemps is:  62.33
===> Visualized output of batchSEN10_1.8_6_300-400_Test_000001_4exemps is saved to ./2025/Outputs/SEN10_1.8_6_300-400_Test_000001_4exemps_out.png
Memory Gain Since Starting:57176064 (Difference: 14712832)
===> The predicted count for SEN10_1.8_6_300-400_Test_000002_4exemps is:  70.27
===> Visualized output of batchSEN10_1.8_6_300-400_Test_000002_4exemps is saved to ./2025/Outputs/SEN10_1.8_6_300-400_Test_000002_4exemps_out.png
Memory Gain Since Starting:-5550080 (Difference: -62726144)
===> The predicted count for SEN10_1.8_6_300-400_Test_000003_4exemps is:  91.98
===> Visualized output of batchSEN10_1.8_6_300-400_Test_000003